<a id="top"></a>

The purpose of this notebook is to go through the process of making a sample prediction using the route model approach.

***

# Import Packages

In [1]:
import requests
import traceback
import json
import pickle
import datetime

***

<a id="contents"></a>
# Contents

- [1. Stop Travel Time %'s](#1)
- [2. Model Prediction](#2)
- [3. Total Dwell Time](#3)
- [4. Calculate Journey Time](#4)

***

# Chosen Route

UCD (N11 entrance) to Trinity College (Dawson Street).  
Route 39A, stops 768 to 793.

In [2]:
# set line and stops
line = '39A'
origin = '768'
dest = '793'

For this calculation we need the following variables:

- dest_pct
- origin_pct
- model_prediction
- total_dwelltime

***

<a id="1"></a>
# 1. Stop Travel Time %'s
[Back to contents](#contents)

## 1.1. Load Stop Time % Dictionary

In [3]:
# load stop time percent dictionary
f = open('/home/faye/Data-Analytics-CityRoute/data/stop_time_pct.json',)

stop_times = json.load(f)

## 1.2. Get Direction

In [4]:
# get direction of route
stops = list(stop_times[line]['outbound'].keys())

origin_index = stops.index(origin)
dest_index = stops.index(dest)

if origin_index < dest_index:
    direction = 'outbound'
else:
    direction = 'inbound'
    
print(f"Direction: {direction}")

Direction: outbound


## 1.3. Set % Variables

In [5]:
# set pct variables
# divide by 100 to convert to percentage
dest_pct = stop_times[line][direction][dest] / 100
origin_pct = stop_times[line][direction][origin] / 100

print(f"% of the total route for origin stop:      {origin_pct:.2}")
print(f"% of the total route for destination stop: {dest_pct:.2}")

% of the total route for origin stop:      0.027
% of the total route for destination stop: 0.27


<a id="2"></a>
# 2. Model Prediction
[Back to contents](#contents)

For the model we need the following parameters

parameters = [

    'HOUR',
    
    'DAYOFWEEK_Monday', <- dropped dummy
    'DAYOFWEEK_Tuesday',
    'DAYOFWEEK_Wednesday',
    'DAYOFWEEK_Thursday',
    'DAYOFWEEK_Friday',
    'DAYOFWEEK_Saturday',
    'DAYOFWEEK_Sunday',
       
    'MONTHOFSERVICE_January', <- dropped dummy
    'MONTHOFSERVICE_February',
    'MONTHOFSERVICE_March',
    'MONTHOFSERVICE_April',
    'MONTHOFSERVICE_May',
    'MONTHOFSERVICE_June',
    'MONTHOFSERVICE_July',
    'MONTHOFSERVICE_August',
    'MONTHOFSERVICE_September',
    'MONTHOFSERVICE_October',
    'MONTHOFSERVICE_November',
    'MONTHOFSERVICE_December',
    
    'IS_HOLIDAY_0', <- dropped dummy
    'IS_HOLIDAY_1',
    
    'humidity',
    'rain_1h',
    'temp',
    'wind_speed',
    
    'weather_main_Clear', <- dropped dummy
    'weather_main_Clouds',
    'weather_main_Drizzle',
    'weather_main_Fog',
    'weather_main_Mist',
    'weather_main_Rain',
    'weather_main_Smoke',
    'weather_main_Snow',
    
]

## 2.1. Load in Model

In [6]:
# set model name using line and direction
model_name = f"route_{line}_{direction}"

In [7]:
# set file path
file_path = f"/home/faye/Data-Analytics-CityRoute/route_models/{model_name}.pkl"

In [8]:
# load the model
linear_reg = pickle.load(open(file_path, 'rb'))

## 2.2. Set Calendar Variables

Take the user input as 09/08/2021 at 09:00

In [9]:
# set hour (from user input)
hour = 9

In [10]:
# set day
dayofweek = [0,0,0,0,0,0]

In [11]:
# set month
monthofservice = [0,0,0,0,0,0,0,0,0,0,0]

# month is August (at index 6)
monthofservice[6] = 1

In [12]:
# set is_holiday - selected day isn't a holiday
is_holiday = 0

## 2.3. Set Weather Variables

In [13]:
# Query OWM for forecasted weather
WEATHER = "http://api.openweathermap.org/data/2.5/weather?"
CITY = "Dublin,IE"
OW_APIkey = 'a13b4ad387112eb8e23757d9dcdbc27d'

curr_weather = requests.get(WEATHER, params={"q": CITY, "appid": OW_APIkey, "units": 'metric'})

weather = curr_weather.json()

In [14]:
# set continuous weather variables
humidity = weather['main']['humidity']
try:
    rain_1h = weather['rain']['1h']
except:
    rain_1h = 0
temp = weather['main']['temp']
wind_speed = weather['wind']['speed']

weather_cont = [humidity, rain_1h, temp, wind_speed]

In [15]:
# set weather_main
weather_main = [0, 0, 0, 0, 0, 0, 0]

main = weather['weather'][0]['main']

if main == 'Clouds':
    weather_main[0] = 1
elif main == 'Drizzle':
    weather_main[1] = 1
elif main == 'Fog':
    weather_main[2] = 1
elif main == 'Mist':
    weather_main[3] = 1
elif main == 'Rain':
    weather_main[4] = 1
elif main == 'Smoke':
    weather_main[5] = 1
else:
    weather_main[6] = 1

## 2.4. Make Model Prediction

In [16]:
# set parameters array 
parameters = []

parameters.append(hour)
parameters.extend(dayofweek)
parameters.extend(monthofservice)
parameters.append(is_holiday)
parameters.extend(weather_cont)
parameters.extend(weather_main)

In [17]:
# print parameters array
print("Model Input Parameters:")
print(parameters)

Model Input Parameters:
[9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 77, 0, 17.5, 5.81, 1, 0, 0, 0, 0, 0, 0]


In [18]:
# make prediction (returns predict time for entire route)
model_prediction = linear_reg.predict([parameters])[0]

print(f"Model prediciton: {model_prediction} seconds")

Model prediciton: 4980.349558703355 seconds


***

<a id="3"></a>
# 3. Total Dwell Time
[Back to contents](#contents)

In [19]:
# Load stop_dwelltimes dictionary
f = open('/home/faye/Data-Analytics-CityRoute/data/stop_dwelltimes.json',)

stop_dwelltimes = json.load(f)

In [20]:
# get list of route stops using line and direction
all_stops = list(stop_times[line][direction].keys())

In [21]:
# get all stops on selected journey using origin and dest indices (from above section 1.2.)
journey_stops = all_stops[origin_index:dest_index]

In [22]:
# calculate total dwelltime
total_dwelltime = 0
for stop in journey_stops:
    total_dwelltime += stop_dwelltimes[stop]

print(f"Total dwell time for selected journey: {total_dwelltime} seconds")

Total dwell time for selected journey: 254.26562730816823 seconds


***

<a id="4"></a>
# 4. Calculate Journey Time
[Back to contents](#contents)

For this calculation we need the following variables:

- dest_pct
- origin_pct
- model_prediction
- total_dwelltime

In [23]:
# calculate predicted journey time
predicted_journey_time = (dest_pct * model_prediction) - (origin_pct * model_prediction) + total_dwelltime

In [24]:
# print predicted journey time
print(predicted_journey_time)
print(datetime.timedelta(seconds=predicted_journey_time))

1465.7020064522276
0:24:25.702006


> If a user selected the route UCD to Trinity (stop 768 to 793 on line 39A) for Monday 9 August at 09:00 our model predicts it should take   
24 minutes and 26 seconds.  
A google maps prediction for the same route conditions returns an estimated journey time of 15 minutes.

***

[Back to top](#top)